In [31]:
from graph_tool.all import *
from graph_tool import Graph, GraphView
from graph_tool import centrality
from graph_tool.clustering import local_clustering, global_clustering
import numpy as np

# Load graphs correctly
G1 = graph_tool.load_graph("/Users/ramius/Desktop/CodeVault/01_Project/Work/Susumu/Student_Survey/StudentSurvey_GitDock/data_WS2425/NA/multiplex_graph_w1.gml")
G2 = graph_tool.load_graph("/Users/ramius/Desktop/CodeVault/01_Project/Work/Susumu/Student_Survey/StudentSurvey_GitDock/data_WS2425/NA/multiplex_graph_w2.gml")
G3 = graph_tool.load_graph("/Users/ramius/Desktop/CodeVault/01_Project/Work/Susumu/Student_Survey/StudentSurvey_GitDock/data_WS2425/NA/multiplex_graph_w3.gml")

# Correct the waves list to include G2
waves = [G1, G2, G3]

metrics = {    
    "net_n_comp": lambda net: len(label_components(net, directed=True)[1]),
    "net_largest_cc": lambda net: max(label_components(net, directed=True)[1]),
    "net_density": lambda net: round((net.num_edges() / (net.num_vertices() * (net.num_vertices() - 1))),3) if net.num_vertices() > 1 else 0,

    "net_global_clustering": lambda net: global_clustering(net),
    "net_assortativity": lambda net: assortativity(net, "total"),

    "hist_in": lambda net: vertex_hist(net, "in"),
    "hist_out": lambda net: vertex_hist(net, "out"),

    "node_array_betweenness": lambda net: np.array(list(centrality.betweenness(net)[0])), #node property map
    "edge_array_betweenness": lambda net: np.array(list(centrality.betweenness(net)[1])), #edge property map

    "node_map_betweenness": lambda net: centrality.betweenness(net)[0], #node property map
    "edge_map_betweenness": lambda net: centrality.betweenness(net)[1], #edge property map
    
    "community_map": lambda net: minimize_nested_blockmodel_dl(net)
}

def network_dict_creator(waves):
    network_dict = {}
    
    for i, wave in enumerate(waves, start=1):
        wave_name = f"G{i}"
        wave_dict = {}
        
        type_prop = wave.ep["type"]
        type_values = [type_prop[edge] for edge in wave.edges()]
        
        seen = set()
        unique_types = []
        for val in type_values:
            if val not in seen:
                seen.add(val)
                unique_types.append(val)
        
        for type_val in unique_types:
            # Filter edges by type
            efilt = [t == type_val for t in type_values]
            u = GraphView(wave, efilt=efilt)
            
            # Calculate total degrees for vertex filter
            vlist = u.get_vertices()
            in_deg = u.get_in_degrees(vlist)
            out_deg = u.get_out_degrees(vlist)
            total_deg = [i + o for i, o in zip(in_deg, out_deg)]
            vfilt = [deg > 0 for deg in total_deg]
            u = GraphView(u, vfilt=vfilt)
            
            # Prune to get subgraph without isolated nodes
            filtered_graph = Graph(u, prune=True)
            
            if type_val in ["aquaintance", "friend"]:
                network_key = f"{wave_name}_{type_val}"
                wave_dict[network_key] = filtered_graph
        
        network_dict[wave_name] = wave_dict
    
    return network_dict

def compute_network_metrics(network_dict, metrics):
    metric_data = {metric: {} for metric in metrics.keys()}
    
    for wave_name, networks in network_dict.items():
        for full_relation_name, network in networks.items():
            # Split "G1_friend" -> ("friend", "G1")
            relation_type = "_".join(full_relation_name.split("_")[1:])  # Extract base relation name
            wave_id = full_relation_name.split("_")[0]  # Should match wave_name
            
            for metric, func in metrics.items():
                try:
                    result = func(network)
                    if relation_type not in metric_data[metric]:
                        metric_data[metric][relation_type] = {}
                    metric_data[metric][relation_type][wave_id] = result
                except Exception as e:
                    print(f"Error computing {metric} for {relation_type} ({wave_id}): {e}")
                    metric_data[metric][relation_type][wave_id] = None
    
    return metric_data

# Usage
network_dict = network_dict_creator(waves)
network_data_dict = compute_network_metrics(network_dict, metrics)

***


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np
import os
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
def generate_metric_graphs(network_data_dict, output_dir="graphs"):
    os.makedirs(output_dir, exist_ok=True)
    
    for metric, relations_data in network_data_dict.items():
        if not metric.startswith("net_"):
            continue
            
        # Prepare DataFrame
        data = []
        for relation, wave_data in relations_data.items():
            for wave, val in wave_data.items():
                if isinstance(val, tuple):
                    data.append({
                        "Relation": relation,
                        "Wave": wave,
                        "Value": val[0],
                        "CI": val[1]
                    })
                else:
                    data.append({
                        "Relation": relation,
                        "Wave": wave,
                        "Value": val
                    })
        
        if not data:
            continue
            
        df = pd.DataFrame(data)
        
        # Plotting
        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=df,
            x="Wave",
            y="Value",
            hue="Relation",
            marker="o",
            markersize=8
        )
        
        # Add error bars if CI exists
        if "CI" in df.columns:
            for _, row in df.iterrows():
                plt.errorbar(
                    x=row["Wave"],
                    y=row["Value"],
                    yerr=row["CI"],
                    fmt='none',
                    color="grey",
                    capsize=3
                )
        
        plt.title(f"{metric} Across Waves")
        plt.savefig(f"{output_dir}/{metric}.png")
        plt.close()
    
    return "Graphs generated successfully!"

generate_metric_graphs(network_data_dict)

'Graphs generated successfully!'

In [35]:
def plot_separate_in_out_degrees(network_data_dict, relations, output_dir="degree_plots"):
    os.makedirs(output_dir, exist_ok=True)
    sns.set_theme(style="whitegrid", palette="tab10")
    
    # Get histogram data
    hist_in = network_data_dict.get("hist_in", {})
    hist_out = network_data_dict.get("hist_out", {})
    
    # Line styles for relations
    line_styles = {
        "friend": '-',       # Solid line for "friend"
        "aquaintance": '--',  # Dashed line for "aquaintance"
    }
    
    # Create figure for In-Degree
    plt.figure(figsize=(12, 8))
    markers = ['o', 's', 'D']  # Different markers for different waves
    
    for rel in relations:
        # Get data for current relation
        in_data = hist_in.get(rel, {})
        out_data = hist_out.get(rel, {})
        
        # Get common waves between in and out data
        common_waves = sorted(set(in_data.keys()) & set(out_data.keys()))
        
        for wave_idx, wave in enumerate(common_waves):
            # Get in-degree values
            in_values, in_bins = in_data[wave]
            in_bins = in_bins[:-1]  # Remove last bin edge
            
            # Plot in-degree
            plt.plot(in_bins, in_values,
                     linestyle=line_styles.get(rel, '-'),  # Vary line style by relation
                     marker=markers[wave_idx % len(markers)],
                     label=f"{rel} In ({wave})")
    
    plt.title(f"In-Degree Distributions: {' vs '.join(relations)}")
    plt.xlabel("Degree (log scale)")
    plt.ylabel("Frequency (log scale)")
    plt.xscale('log')
    plt.yscale('log')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    filename_in = f"in_degrees_{'_'.join(relations)}.png"
    plt.savefig(os.path.join(output_dir, filename_in), dpi=300)
    plt.close()

    # Create figure for Out-Degree
    plt.figure(figsize=(12, 8))

    for rel in relations:
        in_data = hist_in.get(rel, {})
        out_data = hist_out.get(rel, {})
        
        # Get common waves between in and out data
        common_waves = sorted(set(in_data.keys()) & set(out_data.keys()))
        
        for wave_idx, wave in enumerate(common_waves):
            # Get out-degree values
            out_values, out_bins = out_data[wave]
            out_bins = out_bins[:-1]
            
            # Plot out-degree
            plt.plot(out_bins, out_values,
                     linestyle=line_styles.get(rel, '--'),  # Vary line style by relation
                     marker=markers[wave_idx % len(markers)],
                     label=f"{rel} Out ({wave})")
    
    plt.title(f"Out-Degree Distributions: {' vs '.join(relations)}")
    plt.xlabel("Degree (log scale)")
    plt.ylabel("Frequency (log scale)")
    plt.xscale('log')
    plt.yscale('log')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    filename_out = f"out_degrees_{'_'.join(relations)}.png"
    plt.savefig(os.path.join(output_dir, filename_out), dpi=300)
    plt.close()
    
    return f"Saved in-degree plot: {filename_in}, out-degree plot: {filename_out}"

# Usage
plot_separate_in_out_degrees(network_data_dict, ["friend", "aquaintance"])


'Saved in-degree plot: in_degrees_friend_aquaintance.png, out-degree plot: out_degrees_friend_aquaintance.png'

In [36]:
def plot_edge_betweenness_kde_dual(metric_data, relation_types=["friend", "aquaintance"], output_dir="betweenness_plots"):
    os.makedirs(output_dir, exist_ok=True)
    
    plt.figure(figsize=(10, 6))
    colors = ['navy', 'darkred', 'darkgreen', 'purple', 'orange']
    linestyles = {
        "friend": "solid",
        "aquaintance": "dashed"  # You can change to "dotted", "dashdot", etc.
    }

    for rel in relation_types:
        if rel not in metric_data["edge_array_betweenness"]:
            print(f"Relation type '{rel}' not found in data.")
            continue

        data = metric_data["edge_array_betweenness"][rel]

        for i, (wave, arr) in enumerate(data.items()):
            arr = np.array(arr)
            if len(arr) == 0:
                continue
            kde = gaussian_kde(arr)
            x_vals = np.linspace(min(arr), max(arr), 200)
            label = f"{wave} ({rel})"
            plt.plot(
                x_vals, kde(x_vals),
                label=label,
                color=colors[i % len(colors)],
                linestyle=linestyles[rel],
                linewidth=2
            )

    plt.title("Edge Betweenness Centrality (Friend & Aquaintance)")
    plt.xlabel("Betweenness")
    plt.ylabel("Counts")
    plt.legend(title="Wave & Relation Type")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the plot
    filename = f"edge_betweenness_{'_'.join(relation_types)}.png"
    plt.savefig(os.path.join(output_dir, filename), dpi=300)
    plt.close()
    
    return f"Saved edge betweenness plot: {filename}"


def plot_node_betweenness_kde_dual(metric_data, relation_types=["friend", "aquaintance"], output_dir="betweenness_plots"):
    os.makedirs(output_dir, exist_ok=True)
    
    plt.figure(figsize=(10, 6))
    colors = ['navy', 'darkred', 'darkgreen', 'purple', 'orange']
    linestyles = {
        "friend": "solid",
        "aquaintance": "dashed"  # You can change to "dotted", "dashdot", etc.
    }

    for rel in relation_types:
        if rel not in metric_data["node_array_betweenness"]:
            print(f"Relation type '{rel}' not found in data.")
            continue

        data = metric_data["node_array_betweenness"][rel]

        for i, (wave, arr) in enumerate(data.items()):
            arr = np.array(arr)
            if len(arr) == 0:
                continue
            kde = gaussian_kde(arr)
            x_vals = np.linspace(min(arr), max(arr), 200)
            label = f"{wave} ({rel})"
            plt.plot(
                x_vals, kde(x_vals),
                label=label,
                color=colors[i % len(colors)],
                linestyle=linestyles[rel],
                linewidth=2
            )

    plt.title("Node Betweenness Centrality (Friend & Aquaintance)")
    plt.xlabel("Betweenness")
    plt.ylabel("Counts")
    plt.legend(title="Wave & Relation Type")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the plot
    filename = f"node_betweenness_{'_'.join(relation_types)}.png"
    plt.savefig(os.path.join(output_dir, filename), dpi=300)
    plt.close()
    
    return f"Saved node betweenness plot: {filename}"

# Usage
plot_edge_betweenness_kde_dual(network_data_dict, relation_types=["friend", "aquaintance"])
plot_node_betweenness_kde_dual(network_data_dict, relation_types=["friend", "aquaintance"])


'Saved node betweenness plot: node_betweenness_friend_aquaintance.png'

In [46]:
def plot_betweenness_centrality(network_dict, network_data_dict, output_dir="betweenness_plots"):
    import os
    from graph_tool.draw import graph_draw
    from graph_tool import centrality
    import matplotlib.cm as cm
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Create colormaps
    vcmap = cm.plasma
    ecmap = cm.viridis
    
    for wave_name in network_dict:
        for network_key in network_dict[wave_name]:
            # Extract relation type and wave ID
            parts = network_key.split('_')
            wave_id = parts[0]
            relation_type = '_'.join(parts[1:])  # Handles multi-word relations
            
            # Get the graph object
            g = network_dict[wave_name][network_key]
            
            try:
                # Retrieve precomputed betweenness values
                node_betweenness = network_data_dict["node_map_betweenness"][relation_type][wave_id]
                edge_betweenness = network_data_dict["edge_map_betweenness"][relation_type][wave_id]
            except KeyError as e:
                print(f"Missing betweenness data for {network_key}: {str(e)}")
                continue
            
            # Normalize values for visualization
            node_norm = node_betweenness.copy()
            edge_norm = edge_betweenness.copy()
            
            if node_norm.a.max() > 0:
                node_norm.a /= node_norm.a.max()
            if edge_norm.a.max() > 0:
                edge_norm.a /= edge_norm.a.max()
            
            # Create output filename
            fname = os.path.join(output_dir, f"{network_key}_betweenness.png")
            
            # Draw and save visualization
            graph_draw(
                g,
                vertex_fill_color=node_betweenness,
                edge_color=edge_betweenness,
                #vertex_size=centrality.prop_to_size(node_norm, mi=5, ma=15),
                #edge_pen_width=centrality.prop_to_size(edge_norm, mi=0.5, ma=3),
                vcmap=vcmap,
                ecmap=ecmap,
                output_size=(1600, 1200),
                output=fname
            )
            
            print(f"Saved visualization: {fname}")
    
    return "Betweenness visualizations generated successfully!"

# Usage
plot_betweenness_centrality(network_dict, network_data_dict)

Saved visualization: betweenness_plots/G1_aquaintance_betweenness.png
Saved visualization: betweenness_plots/G1_friend_betweenness.png
Saved visualization: betweenness_plots/G2_aquaintance_betweenness.png
Saved visualization: betweenness_plots/G2_friend_betweenness.png
Saved visualization: betweenness_plots/G3_aquaintance_betweenness.png
Saved visualization: betweenness_plots/G3_friend_betweenness.png


'Betweenness visualizations generated successfully!'

In [39]:
from graph_tool.all import minimize_nested_blockmodel_dl, draw_hierarchy, prop_to_size
import os

def visualize_communities(graph, output_dir="community_plots", filename="community_plot.png", vertex_prop_name=None):
    os.makedirs(output_dir, exist_ok=True)
    
    # Detect nested community structure
    state = minimize_nested_blockmodel_dl(graph)
    
    # Optional: modify node shape/size based on a vertex property (e.g., gender)
    vprops = {}
    if vertex_prop_name and vertex_prop_name in graph.vp:
        vprop = graph.vp[vertex_prop_name]
        vprops["vertex_shape"] = prop_to_size(vprop, 1, 2, power=1)

    # Draw the community hierarchy
    pos = None  # Let graph-tool pick the best layout
    draw_hierarchy(
        state,
        output=os.path.join(output_dir, filename),
        vertex_text=graph.vertex_index,
        bg_color=[1, 1, 1, 1],  # White background
        output_size=(1000, 1000),
        pos=pos,
        **vprops
    )
    
    print(f"Saved community visualization: {os.path.join(output_dir, filename)}")
    return state

In [44]:
filtered_graph = network_dict["G1"]["G1_friend"]
state = visualize_communities(filtered_graph, filename="G1_friend_community.png", vertex_prop_name="gender")

filtered_graph = network_dict["G2"]["G2_friend"]
state = visualize_communities(filtered_graph, filename="G2_friend_community.png", vertex_prop_name="gender")

filtered_graph = network_dict["G3"]["G3_friend"]
state = visualize_communities(filtered_graph, filename="G3_friend_community.png", vertex_prop_name="gender")

Saved community visualization: community_plots/G1_friend_community.png
Saved community visualization: community_plots/G2_friend_community.png
Saved community visualization: community_plots/G3_friend_community.png
